In [17]:
import tensorflow as tf 

In [21]:
 # 使用tf来求解梯度很简单:
x = tf.constant(3.)
w = tf.constant(5.)
y = x*w
with tf.GradientTape() as tape:# 这里注意tf.GradientTape后边加括号
    tape.watch(w)
    # 这里针对所有的静态变量(非Variable的变量),都需要watch, 如果不想watch, 就必须把这个变量定义为Vatiable的类型
    # 最好加上watch这一句,针对需要求导的参数都进行观察
    # 在这个tape下构建计算图, 并且计算loss
# 就可以用[w_grad] = tape.gradient(loss.[w])这样的公式进行计算梯度
tape.gradient(y,[w])
# 这里因为y和x之间的计算没有被包括在tape之内, 所以梯度求解结果是None
# 这个梯度求解方法只能求解一次, 求解之后系统会自动释放一些资源
# 为了是tape能够多次调用梯度求解结果,可以在tf.GradientTape()的参数中加上persistent = Ture的参数
# 但是这样比较耗费显存, 建议不采用这种办法
# 梯度计算代码不应该在tape之内

[None]

In [22]:
x = tf.constant(3.)
w = tf.constant(5.)
with tf.GradientTape() as tape1:
    tape1.watch(w)
    y2 = w*x
tape1.gradient(y2,[w])

[<tf.Tensor: id=51, shape=(), dtype=float32, numpy=3.0>]

In [ ]:
# 二阶梯度的计算方式

In [28]:
x = tf.Variable(3.)
w = tf.Variable(5.)
b = tf.Variable(4.)
with tf.GradientTape() as t1:
    with tf.GradientTape() as t2:
        yn = w*x +b
    d_w,d_b = t2.gradient(yn,[w,b])
    print(d_w,d_b)
dd_w = t1.gradient(d_w,w)
print(dd_w)

tf.Tensor(3.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
None


In [29]:
# 激活函数以及其梯度
# sigmoid 梯度计算
# grad_s = s*(1-s)

In [34]:
s = tf.linspace(-10.,10.,10)
with tf.GradientTape() as tape3:
    tape3.watch(s)
    y = tf.sigmoid(s)
tape3.gradient(y,[s])

[<tf.Tensor: id=176, shape=(10,), dtype=float32, numpy=
 array([4.5386874e-05, 4.1860685e-04, 3.8362255e-03, 3.3258751e-02,
        1.8632649e-01, 1.8632641e-01, 3.3258699e-02, 3.8362255e-03,
        4.1854731e-04, 4.5416677e-05], dtype=float32)>]

In [ ]:
# tanh激活函数
# 

In [37]:
s1 = tf.linspace(-10.,10.,10)
with tf.GradientTape() as tape4:
    tape4.watch(s1)
    y1 = tf.tanh(s1)
tape4.gradient(y1,[s1])

[<tf.Tensor: id=193, shape=(10,), dtype=float32, numpy=
 array([0.0000000e+00, 7.1525562e-07, 5.9722963e-05, 5.0775758e-03,
        3.5285267e-01, 3.5285220e-01, 5.0775758e-03, 5.9722963e-05,
        7.1525562e-07, 0.0000000e+00], dtype=float32)>]

In [38]:
# ReLU激活函数,很大程度上解决了梯度弥散和梯度爆炸的问题

In [42]:
s2 = tf.linspace(-10.,10.,10)
with tf.GradientTape(persistent=True) as tape5:
    tape5.watch(s2)
    y2 = tf.nn.relu(s2)
    y3 = tf.nn.leaky_relu(s2)
    # leaky_relu与relu的区别就是在输入值小于0的时候回返回一个很小的非0的梯度
print(tape5.gradient(y2,[s2]))
print(tape5.gradient(y3,[s2]))

[<tf.Tensor: id=235, shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>]
[<tf.Tensor: id=239, shape=(10,), dtype=float32, numpy=array([0.2, 0.2, 0.2, 0.2, 0.2, 1. , 1. , 1. , 1. , 1. ], dtype=float32)>]


In [ ]:
# Loss的梯度
# 1.mean squared error均方差损失函数(mse)
# 2.cross entropy error损失函数


In [47]:
i = tf.random.normal([2,4])
w_1 = tf.random.normal([4,3])
b_1 = tf.zeros([3])
y = tf.constant([2,0])# 用于对比的真实label

In [50]:
with tf.GradientTape() as tape6:
    tape6.watch([w_1,b_1])
    prob = tf.nn.softmax(i@w_1 + b_1,axis=1)
    loss = tf.reduce_sum(tf.losses.MSE(tf.one_hot(y,depth = 3),prob))
grads = tape6.gradient(loss,[w_1,b_1])
grads[0]

<tf.Tensor: id=329, shape=(4, 3), dtype=float32, numpy=
array([[ 0.10581937, -0.27172017,  0.16590075],
       [-0.1837901 ,  0.27098364, -0.08719351],
       [-0.11029556,  0.25394598, -0.14365038],
       [-0.16678412,  0.21020497, -0.04342084]], dtype=float32)>

In [51]:
grads[1]

<tf.Tensor: id=328, shape=(3,), dtype=float32, numpy=array([-0.12350981,  0.24501257, -0.12150272], dtype=float32)>

In [55]:
# 计算softmax输出的值的交叉熵损失函数的梯度
i_1 = tf.random.normal([2,4])
w_2 = tf.random.normal([4,3])
b_2 = tf.zeros([3])
y_1 = tf.constant([2,0])# 用于对比的真实label
with tf.GradientTape() as tape7:
    tape7.watch([w_2,b_2])
    logits = i_1@w_2 + b_2
    loss = tf.reduce_sum(tf.losses.categorical_crossentropy(tf.one_hot(y,depth = 3),logits,from_logits = True))
    # 这里一步直接包含了softmax的计算, 这样计算相比分开计算softmax和损失函数的, 更加稳定可靠
grads = tape7.gradient(loss,[w_2,b_2])
grads[0]

<tf.Tensor: id=509, shape=(4, 3), dtype=float32, numpy=
array([[-0.33004153, -0.04946045,  0.379502  ],
       [-0.5197544 , -0.24553359,  0.76528794],
       [ 0.14473207, -0.17832686,  0.03359476],
       [-0.3088742 , -0.09931926,  0.40819344]], dtype=float32)>

In [56]:
grads[1]

<tf.Tensor: id=508, shape=(3,), dtype=float32, numpy=array([-0.25347105,  0.68810594, -0.43463483], dtype=float32)>